# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 26 2022 3:00PM,238200,16,Emersa-8066643,"Emersa Waterbox, LLC",Released
1,May 26 2022 2:54PM,238199,19,ACG-2102490198,ACG North America LLC,Released
2,May 26 2022 2:52PM,238198,20,ALUR064558462,Alumier Labs Inc.,Released
3,May 26 2022 2:52PM,238198,20,ALUR241908935,Alumier Labs Inc.,Released
4,May 26 2022 2:52PM,238198,20,ALUR497290054,Alumier Labs Inc.,Released
5,May 26 2022 2:52PM,238198,20,ALUR558233483,Alumier Labs Inc.,Released
6,May 26 2022 2:52PM,238198,20,ALUR834504687,Alumier Labs Inc.,Released
7,May 26 2022 2:52PM,238198,20,ALUR024986885,Alumier Labs Inc.,Released
8,May 26 2022 2:52PM,238198,20,ALU0006939,Alumier Labs Inc.,Released
9,May 26 2022 2:26PM,238195,10,0085717513,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,238194,Released,1
33,238195,Released,11
34,238198,Released,7
35,238199,Released,1
36,238200,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238194,NaN,NaN,1.0
238195,NaN,NaN,11.0
238198,NaN,NaN,7.0
238199,NaN,NaN,1.0
238200,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238108,0.0,0.0,1.0
238121,0.0,1.0,0.0
238126,0.0,0.0,1.0
238129,0.0,0.0,1.0
238133,0.0,8.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238108,0,0,1
238121,0,1,0
238126,0,0,1
238129,0,0,1
238133,0,8,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238108,0,0,1
1,238121,0,1,0
2,238126,0,0,1
3,238129,0,0,1
4,238133,0,8,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238108,,,1
1,238121,,1,
2,238126,,,1
3,238129,,,1
4,238133,,8,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC"
1,May 26 2022 2:54PM,238199,19,ACG North America LLC
2,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.
9,May 26 2022 2:26PM,238195,10,ISDIN Corporation
20,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix
21,May 26 2022 2:21PM,238193,19,"AdvaGen Pharma, Ltd"
22,May 26 2022 1:56PM,238191,10,"Snap Medical Industries, LLC"
23,May 26 2022 1:48PM,238190,15,"Alexso, Inc."
24,May 26 2022 1:47PM,238189,15,"Carwin Pharmaceutical Associates, LLC"
25,May 26 2022 1:31PM,238186,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",,,1
1,May 26 2022 2:54PM,238199,19,ACG North America LLC,,,1
2,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,,,7
3,May 26 2022 2:26PM,238195,10,ISDIN Corporation,,,11
4,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,,,1
5,May 26 2022 2:21PM,238193,19,"AdvaGen Pharma, Ltd",,,1
6,May 26 2022 1:56PM,238191,10,"Snap Medical Industries, LLC",,,1
7,May 26 2022 1:48PM,238190,15,"Alexso, Inc.",,,1
8,May 26 2022 1:47PM,238189,15,"Carwin Pharmaceutical Associates, LLC",,,1
9,May 26 2022 1:31PM,238186,10,ISDIN Corporation,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",1,,
1,May 26 2022 2:54PM,238199,19,ACG North America LLC,1,,
2,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,7,,
3,May 26 2022 2:26PM,238195,10,ISDIN Corporation,11,,
4,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,1,,
5,May 26 2022 2:21PM,238193,19,"AdvaGen Pharma, Ltd",1,,
6,May 26 2022 1:56PM,238191,10,"Snap Medical Industries, LLC",1,,
7,May 26 2022 1:48PM,238190,15,"Alexso, Inc.",1,,
8,May 26 2022 1:47PM,238189,15,"Carwin Pharmaceutical Associates, LLC",1,,
9,May 26 2022 1:31PM,238186,10,ISDIN Corporation,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",1,,
1,May 26 2022 2:54PM,238199,19,ACG North America LLC,1,,
2,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,7,,
3,May 26 2022 2:26PM,238195,10,ISDIN Corporation,11,,
4,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,May 26 2022 2:54PM,238199,19,ACG North America LLC,1.0,NaN,NaN
2,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,7.0,NaN,NaN
3,May 26 2022 2:26PM,238195,10,ISDIN Corporation,11.0,NaN,NaN
4,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 26 2022 3:00PM,238200,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,May 26 2022 2:54PM,238199,19,ACG North America LLC,1.0,0.0,0.0
2,May 26 2022 2:52PM,238198,20,Alumier Labs Inc.,7.0,0.0,0.0
3,May 26 2022 2:26PM,238195,10,ISDIN Corporation,11.0,0.0,0.0
4,May 26 2022 2:21PM,238194,10,Yusen – 3D Matrix,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2381802,56.0,3.0,0.0
15,714518,3.0,13.0,8.0
16,952688,4.0,0.0,0.0
18,476292,4.0,8.0,0.0
19,952672,6.0,1.0,0.0
20,1190854,36.0,2.0,0.0
21,476284,1.0,1.0,0.0
22,238126,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2381802,56.0,3.0,0.0
1,15,714518,3.0,13.0,8.0
2,16,952688,4.0,0.0,0.0
3,18,476292,4.0,8.0,0.0
4,19,952672,6.0,1.0,0.0
5,20,1190854,36.0,2.0,0.0
6,21,476284,1.0,1.0,0.0
7,22,238126,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,56.0,3.0,0.0
1,15,3.0,13.0,8.0
2,16,4.0,0.0,0.0
3,18,4.0,8.0,0.0
4,19,6.0,1.0,0.0
5,20,36.0,2.0,0.0
6,21,1.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,56.0
1,15,Released,3.0
2,16,Released,4.0
3,18,Released,4.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,13.0,0.0,8.0,1.0,2.0,1.0,0.0
Released,56.0,3.0,4.0,4.0,6.0,36.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,13.0,0.0,8.0,1.0,2.0,1.0,0.0
2,Released,56.0,3.0,4.0,4.0,6.0,36.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,13.0,0.0,8.0,1.0,2.0,1.0,0.0
2,Released,56.0,3.0,4.0,4.0,6.0,36.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()